In [3]:
import joblib
import pandas as pd
from python_files.data_preprocessor import preprocess_test
import requests

In [4]:
model = joblib.load('models\lgbm_classifier.pkl')
scaler = joblib.load('models\lgbm_scaler.pkl')
encoder = joblib.load('models\lgbm_encoder.pkl')

In [11]:
url = "http://127.0.0.1:8000/predict"

data = {
  "Transaction_Id": "6561289625",
  "Date_Time": "2021-05-13 21:26:26",
  "Account_Creation_Date": "2021-05-12 21:26:26",
  "Phone_Number": "+92 327-5724582",
  "CNIC": "15352-7462017-7",
  "Name": "Maira Butt",
  "Type": "PAYMENT",
  "Amount": 9839.64,
  "ID_Source": "C1231006815",
  "Old_Balance": 170136.0,
  "New_Balance": 160296.36,
  "Source_State": "Gilgit-Baltistan",
  "Source_City": "Gilgit",
  "Device_Name": "Google Pixel 6",
  "IMEI": "831780764842",
  "KYC_Status": "On-Hold",
  "Service_Charges": 19.67928,
  "Channel": "Credit Card",
  "Remarks": "",
  "ID_Dest": "M1979787155",
  "Dest_State": "Gilgit-Baltistan",
  "Dest_City": "Gilgit"
}


In [12]:
data_fraud = {'Transaction_Id': '5720078885', 'Date_Time': '2021-04-05 18:32:15', "Account_Creation_Date": "2021-04-05 18:30:26",'Phone_Number': '+92 305-3516186', 'CNIC': '63643-6029379-9', 'Name': 'Bilal Qureshi', 'Type': 'TRANSFER', 'Amount': 837192.23, 'ID_Source': 'C653262851', 'Old_Balance': 837192.23, 'New_Balance': 0.0, 'Source_State': 'Azad Kashmir', 'Source_City': 'Rawalakot', 'Device_Name': 'OnePlus 9', 'IMEI': '468898808901590', 'KYC_Status': 'Validated', 'Service_Charges': 1674.38446, 'Channel': 'Bank Transfer', 'Remarks': '', 'ID_Dest': 'C690345380', 'Dest_State': 'Gilgit-Baltistan', 'Dest_City': 'Skardu'}

In [13]:
df = pd.DataFrame([data_fraud])

In [14]:
processed_data = preprocess_test(df,scaler,encoder)

In [16]:
response = requests.post(url, json=data_fraud)
print(response.json())

{'Prediction Model LGBM': [1], 'Probability Model': [99.42], 'Prediction Business Rules': {'BusinessRule1_rule1_n_transactions_m_minutes': False, 'BusinessRule2_rule2_high_risk_cities': False, 'BusinessRule3_rule3_high_risk_region_and_time': False, 'BusinessRule4_rule4_new_account_with_large_transaction': False}, 'Probability Business Rules': 0.0, 'Prediction Statiscal Rules': {'StatisticalRule5_rule5_large_transaction': True, 'StatisticalRule6_rule6_high_frequency_same_source': False, 'StatisticalRule7_rule7_unusual_dest_accounts': False, 'StatisticalRule8_rule8_time_based_spike': False}, 'Probability Statistical Rules': 25.0}


In [5]:
test_case_1 = {
    'Transaction_Id': '262874940',
    'Date_Time': '2025-11-22 03:51:50',
    'Account_Creation_Date': '2025-11-20 03:51:50',
    'Phone_Number': '+92 333 2569874',
    'CNIC': '40734-8820668-3',
    'Name': 'Fariha Mirza',
    'Type': 'Utility Bill Payment',
    'Amount': '1740100.97',
    'ID_Source': 'C12345896',
    'Old_Balance': '9987000.00',
    'New_Balance': '0.00',
    'Source_State': 'Unknown',
    'Source_City': 'Unknown',
    'Device_Name': 'Xiaomi Mi 11',
    'IMEI': '601635731712294',
    'KYC_Status': 'On-Hold',
    'Service_Charges': '34.80',
    'Channel': 'Mobile Payments',
    'Remarks': 'Utility bill payment',
    'ID_Dest': 'M1855643329',
    'Dest_State': 'Balochistan',
    'Dest_City': 'Dera Bugti'
}

In [ ]:
test_case_2 = {
    'Transaction_Id': '263099341',
    'Date_Time': '2025-10-10 14:18:44',
    'Account_Creation_Date': '2025-10-08',
    'Phone_Number': 'Not Provided',
    'CNIC': '42101-2290987-9',
    'Name': 'Salman Khalid',
    'Type': 'Wallet-to-Wallet Transfer',
    'Amount': 2490000.00,
    'ID_Source': 'C1789461102',
    'Old_Balance': 2500000.00,
    'New_Balance': 0.00,
    'Device_Name': 'Infinix Zero X Neo',
    'IMEI': '351760319091001',
    'KYC_Status': 'Pending',
    'Service_Charges': 50.00,
    'Channel': 'Mobile App',
    'Remarks': 'Wallet Transfer',
    'ID_Dest': 'M1187643899',
    'Dest_State': 'Sindh',
    'Dest_City': 'Lyari Town',
    'Source_State': 'Punjab',
    'Source_City': 'Rawalpindi',
    'Is_Fraud': 1,
}

In [18]:
processed_data = preprocess_test(pd.DataFrame([test_case_2]),scaler,encoder)

In [19]:
prediction = model.predict(processed_data)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: Is_Fraud: object

In [32]:
import numpy as np

In [41]:
np.max(prediction)

np.float64(0.981919394960915)

In [34]:
from python_files.Rules import BusinessRules, StatisticalRules

In [46]:
transaction_id = df['Transaction_Id'][0]

In [45]:
transaction_id[0]

'262874940'

In [52]:
df['Account_Creation_Date'] = pd.to_datetime(df['Account_Creation_Date'], errors='coerce')
cutoff = pd.Timestamp.now() - pd.Timedelta(days=10)
flagged = df[(df['Account_Creation_Date'] >= cutoff) & (df['Amount'] > 300000)]

TypeError: '>' not supported between instances of 'str' and 'int'

In [50]:
cutoff

Timestamp('2025-04-07 12:04:05.151558')

TypeError: '>' not supported between instances of 'str' and 'int'

In [47]:
business = BusinessRules(df)
statistical = StatisticalRules(df)

business_results = business.apply_rules(transaction_id)

statistical_results = statistical.apply_rules(transaction_id)


TypeError: '>' not supported between instances of 'str' and 'int'

In [8]:
probabilities = model.predict_proba(processed_data)[:, 1]

# Define your desired threshold
threshold = 0.5  # You can adjust this value as needed

# Convert probabilities to binary class labels based on the threshold
predictions = (probabilities >= threshold).astype(int)

In [9]:
probabilities

array([0.98191939])

In [10]:
predictions

array([1])